In [34]:

from tqdm.notebook import tqdm
from RL_utils import *
from RL_plot_utils import *

In [54]:
pars = pars_loader(N_episodes=int(100000),alpha=0.1,gamma=0.9,eps=0.15,pt=0.7,ph=0.6,growth_range=5,factor_range=10,ci_range=[0,11],ci_g_range=5,ci_range_sim=[3,7],action_set=[0,1,2])

In [61]:
pars

{'N_episodes': 100000,
 'alpha': 0.1,
 'gamma': 0.9,
 'eps': 0.15,
 'pt': 0.7,
 'ph': 0.6,
 'ci_range_sim': [3, 7],
 'growth_range': 5,
 'factor_range': 10,
 'ci_range': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 'ci_g_range': 5,
 'action_set': [0, 1, 2],
 'opt_range': {1: {'Tem': [[3, 4, 5], [2, 6, 7, 8], [1, 9], [0, 10]],
   'Hum': [[6, 7, 8], [2, 3, 4, 5], [1, 9], [0, 10]]},
  2: {'Tem': [[6, 7, 8], [4, 5], [1, 2, 3, 9], [0, 10]],
   'Hum': [[4, 5, 6], [2, 3, 7, 8], [1, 9], [0, 10]]},
  3: {'Tem': [[5, 6, 7], [3, 4, 8], [1, 2, 9], [0, 10]],
   'Hum': [[6, 7, 8], [3, 4, 5], [1, 2, 9], [0, 10]]},
  4: {'Tem': [[6, 7, 8], [4, 5, 9], [1, 2, 3], [0, 10]],
   'Hum': [[7, 8, 9], [4, 5, 6], [1, 2, 3], [0, 10]]}},
 'nstates': (6, 11, 11)}

In [55]:
def growth_state(growth,temp,hum,range_tem,range_hum):
    #At least one red: bad condition to growth -2
    if temp in range_tem[3] or hum in range_hum[3]:
      growth+=-2
      growth=max(growth,0)
    #At least one orange : poor condition to growth -1
    elif temp in range_tem[2] or hum in range_hum[2]:
      growth+=-1
      growth=max(growth,0)
    #At least one yellow : good condition to growth +0
    elif temp in range_tem[1] or hum in range_hum[1]:
     growth+=0
    else:
     #Only green : Optimal condition to growth +1
     growth+=1
     growth=min(growth,5)
    return growth

In [56]:
def next_state(state,action,pars):
    growth,temp,hum = state
      
    range_tem,range_hum=pars['opt_range'].get(growth)['Tem'],pars['opt_range'].get(growth)['Hum']
    
    #Probability not to decay
    if action == 0:
       wt=np.random.choice([-1,0],p=[1-pars['pt'],pars['pt']])
       wh=np.random.choice([-1,0],p=[1-pars['ph'],pars['ph']])
       temp=max(0,min(temp+wt,pars['factor_range']))
       hum=max(0,min(hum+wh,pars['factor_range']))
    if action == 1:
       wh=np.random.choice([-1,0],p=[1-pars['ph'],pars['ph']])
       temp=max(0,min(temp+1,pars['factor_range']))
       hum=max(0,min(hum+wh,pars['factor_range']))
    if action == 2:
       wt=np.random.choice([-1,0],p=[1-pars['pt'],pars['pt']])
       hum=max(0,min(hum+1,pars['factor_range']))
       temp=max(0,min(temp+wt,pars['factor_range']))
        
    growth=growth_state(growth,temp,hum,range_tem,range_hum)  
    
    r=0     
    if growth == 5:
       r=100

    return (growth,temp,hum),r   

In [57]:
def make_episode(policy,pars,x0=None):
    seq_state=[]
    seq_action=[]
    seq_reward=[]
    
    if x0 is not None:
     g,temp,hum=x0
    else:
     g=np.random.choice(np.arange(1,pars['ci_g_range']))  
     temp=np.random.choice(pars['ci_range'])  
     hum=np.random.choice(pars['ci_range'])
    seq_state.append((g,temp,hum))
    while g!=5 and g!=0:
        #eps-greedy 
        if np.random.random()<pars['eps']:
         a=np.random.choice(pars['action_set'])
        else:
         a=policy[seq_state[-1]]
            
        (g,temp,hum),r=next_state((g,temp,hum),a,pars)        
        seq_state.append((g,temp,hum))
        seq_action.append(a)
        seq_reward.append(r)
    
    return seq_state,seq_action,seq_reward 

In [58]:
def GPI_MC(init_p,pars):
 policy=init_p.copy()

 Q = np.zeros((pars['nstates'][0], pars['nstates'][1], pars['nstates'][2], len(pars['action_set'])))
 
 counter_s_a=np.zeros(Q.shape)
 
 for _ in tqdm(range(pars['N_episodes'])):
  seq_state,seq_action,seq_reward=make_episode(policy,pars)
 
  Q,counter_s_a = action_state_value_MC([seq_state],[seq_action],[seq_reward],pars,Q0=Q,counter_s_a_0=counter_s_a)
 
  for s in set(seq_state):
   policy[s]=np.argmax(Q[s[0],s[1],s[2],:])
   
 _,V = recover_policy_V(Q,pars)
 return policy,Q,V,counter_s_a

In [59]:
test_policy=init_policy(pars)
policy_MC,Q_MC,V_MC,counter_s_a=GPI_MC(test_policy.copy(),pars)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [60]:
plot_value_dyn(V_MC , 0, 5,"State value function GPI-MC")

In [21]:
plot_policy_dyn(policy_MC, 0, 5)

In [26]:
#np.save("Sims-GYM/policy_GPI.npy", policy_MC)

In [27]:
st_list=[]
ac_list=[]
re_list=[]
counter=0
for _ in tqdm(range(pars['N_episodes'])):
 seq_state,seq_action,seq_reward=make_episode(policy_MC,pars)
 counter+= 1 if seq_state[-1][0]==5 else 0
 st_list.append(seq_state)
 ac_list.append(seq_action)
 re_list.append(seq_reward)
print(f'Test_policy{counter/pars["N_episodes"]*100}%')

  0%|          | 0/10000 [00:00<?, ?it/s]

Test_policy68.91000000000001%


## TD

In [62]:
def GPI_TD_Sarsa(pars,x0=None):

 Q = np.zeros((pars['nstates'][0], pars['nstates'][1], pars['nstates'][2], 3))
 
 for _ in tqdm(range(pars['N_episodes'])):
      #seq_state,seq_action,seq_reward=make_episode(policy,pars)
      
      #Initialize state
      if x0 is not None:
         g,temp,hum=x0
      else:
         g=np.random.choice(np.arange(1,pars['ci_g_range']))  
         temp=np.random.choice(pars['ci_range'])  
         hum=np.random.choice(pars['ci_range'])
      s=(g,temp,hum)
      #seq_state.append((g,temp,hum))
     
      #Choose A from S using policy derived from Q (eps-greedy) 
      if np.random.random()<pars['eps']:
       a=np.random.choice([0,1,2])
      else:
       a=np.argmax(Q[s[0],s[1],s[2],:])
          
      #Loop for each step of episode
      while s[0]!=5 and s[0]!=0:  
        #Observe R,S'
        s_next,r=next_state(s,a,pars)        
        # seq_state.append((g,temp,hum))
        # seq_action.append(a)
        # seq_reward.append(r)
            
        #Choose A' from S' using policy derived from Q  (eps-greedy)  
        if np.random.random()<pars['eps']:
         a_next=np.random.choice([0,1,2])
        else:
         a_next=np.argmax(Q[s_next[0],s_next[1],s_next[2],:])
     
        Q[s[0],s[1],s[2],a]+=pars['alpha']*(r + pars['gamma']*Q[s_next[0],s_next[1],s_next[2],a_next]-Q[s[0],s[1],s[2],a])
             
        s = s_next
        a = a_next 
 
 return Q.copy()

In [63]:
Q_sarsa=GPI_TD_Sarsa(pars)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [64]:
policy_sarsa,V_sarsa=recover_policy_V(Q_sarsa,pars)

In [65]:
plot_value_dyn(V_sarsa, 0, 5,"State value function GPI-Sarsa")

In [25]:
plot_policy_dyn(policy_sarsa,1, 4)

In [26]:
def GPI_TD_Q(pars,x0=None):

 Q = np.zeros((pars['nstates'][0], pars['nstates'][1], pars['nstates'][2], 3))
 
 for _ in tqdm(range(pars['N_episodes'])):
      
      #Initialize state
      if x0 is not None:
         g,temp,hum=x0
      else:
         g=np.random.choice(np.arange(1,pars['ci_g_range']))  
         temp=np.random.choice(pars['ci_range'])  
         hum=np.random.choice(pars['ci_range'])
      s=(g,temp,hum)
               
      #Loop for each step of episode
      while s[0]!=5 and s[0]!=0: 
          
        #Choose A from S using policy derived from Q  (eps-greedy)  
        if np.random.random()<pars['eps']:
         a=np.random.choice([0,1,2])
        else:
         a=np.argmax(Q[s[0],s[1],s[2],:])
            
        #Observe R,S'
        s_next,r=next_state(s,a,pars)        
            
        Q[s[0],s[1],s[2],a]+=pars['alpha']*(r+pars['gamma']*np.max(Q[s_next[0],s_next[1],s_next[2],:])-Q[s[0],s[1],s[2],a])
     
        s = s_next
 
 return Q.copy()

In [27]:
Q_Q=GPI_TD_Q(pars,x0=(1,3,4))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [28]:
policy_Q_learning,V_Q_learing=recover_policy_V(Q_Q,pars)

In [29]:
plot_value_dyn(V_Q_learing, 0, 5,"State value function GPI-V_Q_learning")

In [30]:
def GPI_TD_Exp_Sarsa(pars,x0=None):

 Q = np.zeros((pars['nstates'][0], pars['nstates'][1], pars['nstates'][2], 3))
 
 for _ in tqdm(range(pars['N_episodes'])):
      #seq_state,seq_action,seq_reward=make_episode(policy,pars)
      
      #Initialize state
      if x0 is not None:
         g,temp,hum=x0
      else:
         g=np.random.choice(np.arange(1,pars['ci_g_range']))  
         temp=np.random.choice(pars['ci_range'])  
         hum=np.random.choice(pars['ci_range'])
      s=(g,temp,hum)
      #seq_state.append((g,temp,hum))
     
      #Choose A from S using policy derived from Q (eps-greedy) 
      if np.random.random()<pars['eps']:
       a=np.random.choice([0,1,2])
      else:
       a=np.argmax(Q[s[0],s[1],s[2],:])
          
      #Loop for each step of episode
      while s[0]!=5 and s[0]!=0:  
        #Observe R,S'
        s_next,r=next_state(s,a,pars)        
        # seq_state.append((g,temp,hum))
        # seq_action.append(a)
        # seq_reward.append(r)
            
        #Choose A' from S' using policy derived from Q  (eps-greedy)  
        if np.random.random()<pars['eps']:
         a_next=np.random.choice([0,1,2])
        else:
         a_next=np.argmax(Q[s_next[0],s_next[1],s_next[2],:])
            
        # probabilities of current policy (eps-greedy)   
        prob = np.ones(3)*pars['eps']/3
        a_greedy = np.argmax(Q[s_next[0],s_next[1],s_next[2],:])
        prob[a_greedy] = 1 - pars['eps'] + pars['eps']/3
        # compute expected value of Q(s_next,a)
        exp_Q = 0
        for act,p in zip(Q[s_next[0],s_next[1],s_next[2],:], prob):
            exp_Q += act*p
        
        Q[s[0],s[1],s[2],a]+=pars['alpha']*(r + pars['gamma']*exp_Q-Q[s[0],s[1],s[2],a])
             
        s = s_next
        a = a_next 
 
 return Q.copy()

In [31]:
Q_exp_sarsa=GPI_TD_Exp_Sarsa(pars,x0=(1,3,4))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [32]:
policy_Exp_Sarsa, V_Exp_Sarsa = recover_policy_V(Q_exp_sarsa, pars)

In [33]:
plot_value_dyn(V_Exp_Sarsa , 0, 5, "State value function GPI-V_Exp_Sarsa")

In [34]:
counter=0
for _ in tqdm(range(pars['N_episodes'])):
 seq_state,seq_action,seq_reward=make_episode(policy_MC,pars,x0=(1,3,4))
 counter+= 1 if seq_state[-1][0]==5 else 0
print(f'MC:{counter/pars["N_episodes"]*100}%')

  0%|          | 0/100000 [00:00<?, ?it/s]

MC:99.456%


In [35]:
counter=0
for _ in tqdm(range(pars['N_episodes'])):
 seq_state,seq_action,seq_reward=make_episode(policy_Exp_Sarsa,pars,x0=(1,3,4))
 counter+= 1 if seq_state[-1][0]==5 else 0
print(f'Exp_Sarsa:{counter/pars["N_episodes"]*100}%')

  0%|          | 0/100000 [00:00<?, ?it/s]

Exp_Sarsa:0.0%


In [36]:
counter=0
for _ in tqdm(range(pars['N_episodes'])):
 seq_state,seq_action,seq_reward=make_episode(policy_Q_learning,pars,x0=(1,3,4))
 counter+= 1 if seq_state[-1][0]==5 else 0
print(f'Q_Learning:{counter/pars["N_episodes"]*100}%')

  0%|          | 0/100000 [00:00<?, ?it/s]

Q_Learning:0.0%


In [37]:
counter=0
for _ in tqdm(range(pars['N_episodes'])):
 seq_state,seq_action,seq_reward=make_episode(policy_sarsa,pars,x0=(1,3,4))
 counter+= 1 if seq_state[-1][0]==5 else 0
print(f'Sarsa:{counter/pars["N_episodes"]*100}%')

  0%|          | 0/100000 [00:00<?, ?it/s]

Sarsa:0.0%


NameError: name 'policy_Q_learning' is not defined

In [ ]:
gsim = 1
tsim = np.random.choice(pars['ci_range_sim']) 
hsim = np.random.choice(pars['ci_range_sim'])